# Titanic corrigé

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
titanic_data = pd.read_csv("http://web.stanford.edu/class/archive/cs/cs109/cs109.1166/stuff/titanic.csv")

## Exo 1

In [ ]:
# pas de valeurs manquantes
titanic_data.info()

In [ ]:
y=titanic_data['Survived']
y.shape

In [ ]:
#545 victimes et 342 survivants
from collections import Counter
Counter(y)


On vérifie la constitution des lignes

In [ ]:
titanic_data.values[0]

In [ ]:
# age moyen des victimes
ages=[]
for line in titanic_data.values:
    if line[0]==0:
        ages.append(line[4])
np.average(ages)

Aparté : mesurer la vitesse d'execution d'une cellule

In [ ]:
%%timeit
# age moyen des victimes
ages=[]
for line in titanic_data.values:
    if line[0]==0:
        ages.append(line[4])
np.average(ages)

In [ ]:
# avec des listes en compréhension

ages=[line[4] for line in titanic_data.values if line[0]==0 ]
np.average(ages)

In [ ]:
ages=[line[4] for line in titanic_data.values if line[0]==1 ]
np.average(ages)

Plus joli, avec indexation par masque booléen

In [ ]:
np.average(titanic_data['Age'][titanic_data['Survived']==1])

Aparté : Pour mesurer la vitesse d'execution d'une seule ligne (avec output). On peut remplacer les underscores par In[cellule précédente] ou écrire directement l'instruction

In [ ]:
%timeit __

## Exo 2
On commence par enlever les attributs non pertinents. ici, le nom des passagers (tous distincts)

In [ ]:
x=titanic_data.drop('Survived', axis=1)

On vérifie 

In [ ]:
x.columns, x.shape

In [ ]:
x=x.drop('Name', axis=1)
x = pd.get_dummies(x, columns=['Sex'])
x.columns


## Exo 3

In [ ]:
from sklearn.tree import DecisionTreeClassifier
 
clf = DecisionTreeClassifier().fit(x,y)

In [ ]:
from sklearn.tree import export_graphviz
from IPython.core.display import Image, display
def printTree(clf):
    with open("tree.dot", 'w') as f:
        export_graphviz(clf, out_file=f,
                       feature_names=x.columns,
                        filled=True)
    !dot -Tpng tree.dot -o tree.png 
    display(Image('tree.png'))
    
printTree(clf)

## Exo 4
On reprend les mesures de performances

In [ ]:
def accuracy(y_pred,y) : 
    return (sum(y_pred == y))/len(y)


In [ ]:
import random
def data_split(x, y, prob):
        train,test=[],[]
        d=zip(x,y)
        for line in d:
            if(random.random()<prob):
                train.append(line)
            else :
                test.append(line)
        
        x_train,y_train=list(zip(*train))
        x_test,y_test=list(zip(*test))
        return x_train,y_train, x_test,y_test


On mesure la performance du classifieur en séparant les données en 2 ensembles d'apprentissage et de tests 

In [ ]:
x_train,y_train, x_test,y_test=data_split(x.values, y, .7)
clf = DecisionTreeClassifier().fit(x_train,y_train)
y_pred=clf.predict(x_test)
print(accuracy(y_pred,y_test))

Pour le calcul de la performance en fonction de la profondeur, on moyenne la performance sur 100 tirages pour avoir une valeur plus fiable

In [ ]:
def calculAccuracy(maxdepth):
    res=[]
    for i in range(100):
        x_train,y_train, x_test,y_test=data_split(x.values, y, .5)
        clf = DecisionTreeClassifier(max_depth=maxdepth).fit(x_train,y_train)
        y_pred=clf.predict(x_test)
        res.append(accuracy(y_pred,y_test))
    return np.average(res)

calculAccuracy(20)

L'arbre complet compte 22 niveaux

In [ ]:
accTest=[calculAccuracy(maxdepth) for maxdepth in np.arange(1,23) ]

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns 

plt.ylim([0,1])
plt.plot(accTest);

In [ ]:
def calculAccuracyTRAIN(maxd):
    res=[]
    for i in range(100):
        x_train,y_train, x_test,y_test=data_split(x.values, y, .7)
        clf = DecisionTreeClassifier(max_depth=maxd).fit(x_train,y_train)
        y_pred=clf.predict(x_train)
        res.append(accuracy(y_pred,y_train))
    return np.average(res)

In [ ]:
accTrain=[calculAccuracyTRAIN(maxdepth) for maxdepth in np.arange(1,23) ]

In [ ]:
plt.plot(accTest)
plt.plot(accTrain);

On constate que la performance est maximisée avec max_depth = 3. On obtient donc l'arbre suivant  :


In [ ]:
x_train,y_train, x_test,y_test=data_split(x.values, y, .7)
clf = DecisionTreeClassifier(max_depth=3).fit(x_train,y_train)
printTree(clf)

De même, on peut étudier l'impact d'autres (hyper) - paramètres sur la performance : 

- max_leaf_nodes
- min_samples_leaf
- min_samples_split

Attention, si on souhaite combiner (naivement) la recherche de paramètres optimaux, il faudrait imbriquer autant de boucles que de paramètres à tester. 

## Exo 5
Répartition homme/femme chez les rescapés. 
D'abord la repartition globale :

In [ ]:
Counter(titanic_data['Sex'])

Ensuite, la répartition selon la destiné

In [ ]:
surv= titanic_data['Sex'][titanic_data['Survived']==1]
noy= titanic_data['Sex'][titanic_data['Survived']==0]

In [ ]:
csurv=Counter(surv)
cnoy=Counter(noy)

In [ ]:
csurv['female'],cnoy['female']

In [ ]:
csurv['male'],cnoy['male']

On peut aussi utiliser la méthode pd.crosstab

In [ ]:
table = pd.crosstab(titanic_data['Survived'],titanic_data['Sex'])
table

In [ ]:
# pour les barres
plt.bar(range(len(csurv)), csurv.values(), align='center' )
# pour l'échelle sous les barres
plt.xticks(range(len(csurv)),  csurv.keys())
# légende
plt.title('Survivants');


In [ ]:
plt.bar(range(len(cnoy)), cnoy.values(), align='center' )
plt.xticks(range(len(cnoy)),  cnoy.keys())
plt.title('Victimes');

In [ ]:
fig,ax=plt.subplots(nrows=1, ncols=2,figsize=(20, 5))
ax[0].bar(range(2), Counter(cnoy).values(), align='center')
ax[1].bar(range(2), Counter(csurv).values(), align='center')
ax[0].set_title('Victimes')
ax[1].set_title('Survivants')

plt.sca(ax[0])
plt.xticks(range(2), cnoy.keys(), color='red')
plt.sca(ax[1])
plt.xticks(range(2), cnoy.keys(), color='green');

 

Pour finir, exporter ce notebook en pdf (installer latex au besoin)